In [46]:
from pyscf import gto, scf, cc

a = 2 # bond length in a cluster
d = 100 # distance between each cluster
unit = 'b' # unit of length
na = 2  # size of a cluster (monomer)
nc = 2 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = '6-31g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms,basis=basis,spin=spin*nc,unit=unit,verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
mf.kernel()

nfrozen = nc*frozen

mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Mon Dec 15 13:49:09 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = b
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr  

np.float64(-0.0680054623488666)

In [47]:
# PM localization
import numpy as np
from pyscf import lo
orbocc = mf.mo_coeff[:,nfrozen:np.count_nonzero(mf.mo_occ)]
mlo = lo.PipekMezey(mol, orbocc)
lo_coeff = mlo.kernel()
while True: # always performing jacobi sweep to avoid trapping in local minimum/saddle point
    lo_coeff1 = mlo.stability_jacobi()[1]
    if lo_coeff1 is lo_coeff:
        break
    mlo = lo.PipekMezey(mf.mol, lo_coeff1).set(verbose=4)
    mlo.init_guess = None
    lo_coeff = mlo.kernel()




******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 1  delta_f= 1  |g|= 0.00399999  1 KF 1 Hx
macro= 2  f(x)= 1  delta_f= 0  |g|= 1.73334e-08  1 KF 1 Hx
macro X = 2  f(x)= 1  |g|= 1.73334e-08  4 intor 2 KF 2 Hx


In [59]:
# Fragment list: for PM, every orbital corresponds to a fragment
frag_lolist = [[i] for i in range(lo_coeff.shape[1])]
from pyscf.lno import lnoccsd
thresh = 1e-4
mcc = lnoccsd.LNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=frozen).set(verbose=5)
mcc.lno_thresh = [thresh*10,thresh]
mcc.kernel()


******** <class 'pyscf.lno.lnoccsd.LNOCCSD_T'> ********
nocc = 2, nmo = 8
frozen orbitals 0
max_memory 4000 MB (current use 199 MB)
nfrag = 2  nlo = 2
frag_lolist = [[0], [1]]
frag_wghtlist = None
lno_type = ['1h', '1h']
lno_thresh = [0.001, 0.0001]
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
verbose_imp = 2
_ovL = None
_ovL_to_save = None
force_outcore_ao2mo = False
_match_oldcode = False
_max_las_size_ccsd = 1000
_max_las_size_ccsd_t = 1000
Regularized frag_wghtlist = [1. 1.]
    CPU time for LO and fragment        0.00 sec, wall time      0.00 sec

WARN: Input vhf is not found. Building vhf from SCF MO.

ao2mo est mem= 0.01 MB  avail mem= 3800.55 MB
aux blksize for incore ao2mo: 92/92
    CPU time for Integral xform         0.01 sec, wall time      0.00 sec
LO occ proj: 1 active | 0 standby | 1 orthogonal
LO vir proj: 0 active | 0 standby | 6 orthogonal
    CPU time for Fragment 1 make las      0.02 sec, wall time      0.00 sec
Fragment 1/2

np.float64(-0.06800546128574131)

In [ ]:
# def kernel(mlno, lo_coeff, frag_lolist, lno_type, lno_thresh=None, lno_pct_occ=None,
#            lno_norb=None, eris=None):
#     r''' Kernel function for LNO-based methods.

#     Args:
#         lo_coeff (np.ndarray):
#             Column vectors are the AO coefficients for a set of local(ized) orbitals.
#             These LOs must span at least the occupied space but can span none, part, or
#             full of the virtual space. Thus, lo_coeff.shape[1] >= nmo.
#         frag_lolist (list of list):
#             Fragment definition in terms of the LOs specified by 'lo_coeff'. E.g.,
#                 [[0,1,2],[3,5],[4],[6,7,8,9]...]
#             means
#                 fragment 1 consists of LO 0, 1, and 2,
#                 fragment 2 consists of LO 3 and 5,
#                 fragment 3 consists of LO 4,
#                 fragment 4 consists of LO 6, 7, 8, and 9,
#                 ...
#     '''

lo_coeff = lo_coeff
lno_thresh = [1e-4, 1e-5]
lno_type = ['1h','1h']

frag_lolist = [[0]]

mlno = lnoccsd.LNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=5)

mlno.lno_type = ['1h','1h'] # if lno_type is None else lno_type
mlno.lno_thresh = lno_thresh # if lno_thresh is None else lno_thresh
mlno.lno_pct_occ = None
mlno.lno_norb = None
mlno.lo_proj_thresh = 1e-10
mlno.lo_proj_thresh_active = 0.0001
mlno.eris = None

nfrag = len(frag_lolist)
# if lno_pct_occ is None:
lno_pct_occ = [None, None]
# if lno_norb is None:
lno_norb = [[None,None]] * nfrag
mf = mlno._scf

eris = mlno.ao2mo()

# Loop over fragment
frag_res = [None] * nfrag
for ifrag,loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)


ao2mo est mem= 0.01 MB  avail mem= 3808.37 MB
aux blksize for incore ao2mo: 92/92
    CPU time for Integral xform         0.00 sec, wall time      0.00 sec
LO occ proj: 1 active | 0 standby | 1 orthogonal
LO vir proj: 0 active | 0 standby | 6 orthogonal


In [63]:
uocc_loc.shape

(1, 1)

In [62]:
print(lno_frozen)
print(uocc_loc)
print(lno_coeff)

[0 5 6 7]
[[-1.]]
[[-2.79466478e-01 -3.02995851e-20 -1.66390354e-16  1.72577572e-23
   3.31883915e-16  1.96430288e-01 -8.63954001e-01 -9.79357974e-01]
 [-3.43692715e-01 -3.72629545e-20 -1.09006965e-15  1.16757915e-22
   2.17426414e-15  1.12676762e+00  6.70970414e-01  1.22462965e+00]
 [-2.79466483e-01 -3.02995856e-20  1.66390299e-16 -1.82547831e-23
  -3.31883804e-16 -2.00226703e-01 -9.04250305e-01  9.41479720e-01]
 [-3.43692822e-01 -3.72629661e-20  1.09006965e-15 -1.15969292e-22
  -2.17426414e-15 -1.12376471e+00  7.27628360e-01 -1.19466904e+00]
 [ 3.02995856e-20 -2.79466483e-01 -1.75999785e-01  8.84307086e-01
  -9.64970707e-01  1.44063164e-15 -3.74520929e-18  3.63246112e-17]
 [ 3.72629661e-20 -3.43692822e-01 -1.15302407e+00 -6.99461483e-01
   1.18355963e+00 -3.06094167e-15  8.01337813e-18 -7.45003781e-17]
 [ 3.02995851e-20 -2.79466478e-01  1.75999844e-01  8.84306960e-01
   9.64970813e-01 -1.44063196e-15  3.74520618e-18 -3.63246180e-17]
 [ 3.72629545e-20 -3.43692715e-01  1.15302407e+00 -

In [60]:
mlno.impurity_solve(mf, lno_coeff, uocc_loc, eris, frozen=lno_frozen)

    impsol:  1 LOs  4/8 MOs  1 occ  3 vir


((NPArrayWithTag(-0.02119452),
  NPArrayWithTag(-0.03400273),
  np.float64(5.597755243434074e-20)),
 'E_corr(MP2) = -0.0211945193715294  E_corr(CCSD) = -0.0340027306436184  E_corr(CCSD(T)) = 5.59775524343407e-20')

In [61]:
mo_occ = mlno.mo_occ
lno_frozen, maskact = lnoccsd.get_maskact(lno_frozen, mo_occ.size)
mcc = lnoccsd.CCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=4)
lnoccsd.impurity_solve(mcc, lno_coeff, uocc_loc, mo_occ, maskact, eris)

Init t2, MP2 energy = -2.18882224691047  E_corr(MP2) -0.0211945193715294

******** <class 'pyscf.lno.lnoccsd.MODIFIED_CCSD'> ********
CC2 = 0
CCSD nocc = 1, nmo = 4
frozen orbitals [0 5 6 7]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 199 MB)
Init E_corr(MODIFIED_CCSD) = -0.0211945193715294
cycle = 1  E_corr(MODIFIED_CCSD) = -0.0285603980858457  dE = -0.00736587871  norm(t1,t2) = 0.0417799
cycle = 2  E_corr(MODIFIED_CCSD) = -0.0315725971127073  dE = -0.00301219903  norm(t1,t2) = 0.0189999
cycle = 3  E_corr(MODIFIED_CCSD) = -0.0340212139537715  dE = -0.00244861684  norm(t1,t2) = 0.00875621
cycle = 4  E_corr(MODIFIED_CCSD) = -0.033999436591048  dE = 2.17773627e-05  norm(t1,t2) = 0.000203218
cycle = 5  E_corr(MODIFIED_CCSD) = -0.0340037339448324  dE = -4.29735378e-06  norm(t1,t2) = 1.91681e-05
cycle = 6  E_corr(MODIFIED_CCSD) = -0.0340026500991267  dE = 1.08384571e-06  

((NPArrayWithTag(-0.02119452), NPArrayWithTag(-0.03400273), 0.0),
 'E_corr(MP2) = -0.0211945193715294  E_corr(CCSD) = -0.0340027306436193  E_corr(CCSD(T)) = 0')

In [66]:
prjlo = uocc_loc @ uocc_loc.T.conj()

# print(f'# LNO-MP2 Orbital Energy: {eorb_mp2_cc[ifrag][0]:.8f}')
# print(f'# LNO-CCSD Orbital Energy: {eorb_mp2_cc[ifrag][1]:.8f}')
# print(f'# LNO-CCSD(T) Orbital Energy: {eorb_mp2_cc[ifrag][2]:.8f}')
# print(f'# LNO-CCSD t2=0 Orbital Energy: {elcorr_t1:.8f}')

# from mpi4py import MPI
# if not MPI.Is_finalized():
#     MPI.Finalize()
options = {'n_eql': 3,
           'n_prop_steps': 50,
            'n_ene_blocks': 1,
            'n_sr_blocks': 1,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'ccsd_pt2_ad',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            "max_error":1e-4
            }
# options["seed"] = seeds[ifrag]
from ad_afqmc.lno_afqmc.ccsd_pt2 import lno_afqmc
nelec, norb \
    = lno_afqmc.prep_afqmc(mf,lno_coeff,mcc.t1,mcc.t2,lno_frozen,prjlo,options,chol_cut=1e-5)

# Calculating Effective Active Space One-electron Integrals
# Generating Cholesky Integrals
# Composing AO ERIs from DF basis
# Composing active space MO ERIs from AO ERIs
# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-05
# Finished calculating Cholesky integrals
# Size of the correlation space
# Number of electrons: (1, 1)
# Number of basis functions: 4
# Cholesky shape: (10, 4, 4)


In [6]:
from ad_afqmc.lno_afqmc.ccsd_pt2 import lno_afqmc
options = {'n_eql': 3,
           'n_prop_steps': 50,
            'n_ene_blocks': 1,
            'n_sr_blocks': 5,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'ccsd_pt2_ad',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            "max_error":1e-4
            }
lno_afqmc.run_lnoafqmc(options,nproc=1)

# running AFQMC on CPU
# AFQMC script: /home/sharmagroup/sharmagroup/cs_afqmc/ad_afqmc/lno_afqmc/ccsd_pt2/run_afqmc.py
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 4
# nelec: 2
#
# n_eql: 3
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 5
# n_blocks: 10
# n_walkers: 3
# seed: 2
# walker_type: rhf
# trial: ccsd_pt2_ad
# dt: 0.005
# free_projection: False
# use_gpu: False
# max_error: 0.0001
# n_exp_terms: 6
# ene0: 0.0
# n_batch: 1
#
# <Hbar> = -2.167607, <H> = -2.167628
# 

# Propagating with 3 walkers
# Initial energy -2.167628
# All brakets are measured with HF Trial
# Equilibration sweeps:
#   Iter 	 Energy_hf 	 <exp(T1)H>_orb 	    <T2H>_orb 	 <T2>_orb 	 <H_bar> Ept_orb 	 time
      0 	 -2.167628 	 -2.167607 	 -0.034003 	  0.000000 	 -2.167607 	 -2.201610 	   0.85
      1 	 -2.196589 	 -2.197392 	  -0.042260 